In [ ]:
drive_path = '/content/drive/MyDrive/project3/'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#/content/drive/MyDrive/project3/dump/주형준_vocab/wpm-vocab-je.txt

In [ ]:
import pandas as pd
df_train = pd.read_csv(drive_path + '전처리된_AI_HUB_data/Train/전처리된_AI_hub_train.csv')
df_train.dropna(inplace=True)
df_dev = pd.read_csv(drive_path + '전처리된_AI_HUB_data/Val/전처리된_AI_hub_val.csv')
df_dev.dropna(inplace=True)

In [ ]:
len(df_train)

ai-hub 데이터는 dialect와 standard가 같은 경우가 있다. 둘이 다른 경우만 사용할 수 있도록 데이터를 선별하자.

In [ ]:
condition = df_train['dialect'] != df_train['standard']
df_train = df_train[condition]

In [ ]:
len(df_train)

In [ ]:
df_train[:10]

In [ ]:
df_train_standard = df_train['standard']
df_train_dialect = df_train['dialect']

test_df_train_standard = list(df_train_standard[:200000])
test_df_train_dialect = list(df_train_dialect[:200000])

In [ ]:
print(len(test_df_train_dialect), len(test_df_train_standard))

In [ ]:
test_df_train_standard[:10]

In [ ]:
test_df_train_dialect[:10]

아래아 추가하기

In [ ]:
src_tokenizer = BertTokenizer(drive_path + 'jit_for_AreA_train/dump/wpm-vocab-je_trans.txt', do_lower_case=False)
trg_tokenizer = BertTokenizer(drive_path + 'jit_for_AreA_train/dump/wpm-vocab-je.txt', do_lower_case=False)

In [ ]:
model = EncoderDecoderModel.from_pretrained(drive_path + 'jit_for_AreA_train/dump/model1').to(device)

In [ ]:
model.config.decoder_start_token_id = trg_tokenizer.cls_token_id
model.config.pad_token_id = trg_tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size
model.eval()

In [ ]:
def get_prediction(text):
    embeddings = src_tokenizer(text, return_attention_mask = False, return_token_type_ids= False, return_tensors = 'pt').to(device)
    output = model.generate(**embeddings, max_length = 256, eos_token_id=3)[0, 1:-1].to(device)
    return trg_tokenizer.decode(output[1:])

In [ ]:
from tqdm import tqdm

ai_test_pred = []

for i in tqdm(range(len(test_df_train_dialect))):
    src = test_df_train_dialect[i]
    output = get_prediction(src)
    ai_test_pred.append(output)

In [ ]:
# 데이터 저장하기
import pickle
with open('ai_hub_trans.pkl', 'wb') as f:
    pickle.dump(ai_test_pred, f)

In [ ]:
# 데이터 저장하기
with open('ai_hub_standard.pkl', 'wb') as f:
    pickle.dump(test_df_train_standard, f)